# Beispiel 3 Transfermarkt (by Reto)

Ziel ist eine Tabelle mit den Spalten "Saison" / "Club" / "Gesamtwert" für jede Saison. Gesamtwert bedeutet der Wert einer gesamten Mannschaft und steht für die Kaufkragt eines Clubs.  


Vorgehen: In einem ersten Schritt ein Script bauen für die Saison 2018. In einem zweiten Schritt eine Abfrage für alle Saisons.

## Liste mit den Vereinen erstellen

In [15]:
# Library Import

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
r = requests.get('https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id/2018')
soup = BeautifulSoup(r.text, 'html.parser')
print(r)

<Response [404]>


In [17]:
# ACHTUNG die Webseite kann nicht mit der normalen requests-Funktion abgefragt werden. es muss umgangen werden.

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2018'
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
print(r)

<Response [200]>


In [19]:
vereine = soup.find_all('td', {'class':'hauptlink no-border-links show-for-small show-for-pad'})

In [20]:
vereine[0].text

'Bayern München'

In [22]:
for team in vereine:
    print(team.text)

Bayern München
Bor. Dortmund
Bay. Leverkusen
RB Leipzig
FC Schalke 04
TSG Hoffenheim
Bor. M'gladbach
VfB Stuttgart
VfL Wolfsburg
E. Frankfurt
Hertha BSC
Werder Bremen
1.FSV Mainz 05
FC Augsburg
Hannover 96
SC Freiburg
1.FC Nürnberg
F. Düsseldorf


# Liste mit dem Gesamtwert erstellen

In [30]:
beträge = soup.find_all('td', {'class':'rechts show-for-small show-for-pad nowrap'})

In [31]:
beträge[0].text

'3,79 Mrd. €'

In [34]:
for wert in beträge:
    print(wert.text)

3,79 Mrd. €
7,26 Mio. €
827,80 Mio. €
35,99 Mio. €
412,30 Mio. €
14,22 Mio. €
377,20 Mio. €
14,51 Mio. €
316,00 Mio. €
13,17 Mio. €
256,40 Mio. €
9,16 Mio. €
214,85 Mio. €
7,16 Mio. €
209,65 Mio. €
6,99 Mio. €
165,10 Mio. €
6,88 Mio. €
148,25 Mio. €
4,94 Mio. €
148,00 Mio. €
4,11 Mio. €
135,53 Mio. €
3,87 Mio. €
118,40 Mio. €
3,59 Mio. €
101,95 Mio. €
3,29 Mio. €
100,80 Mio. €
3,25 Mio. €
91,50 Mio. €
3,52 Mio. €
85,65 Mio. €
2,95 Mio. €
41,60 Mio. €
1,54 Mio. €
38,95 Mio. €
1,30 Mio. €


In [35]:
# Es werden zu viele Werte ausgelesen. Deshalb muss der Code adaptiert werden

for wert in beträge[2:][::2]:
    print(wert.text)

827,80 Mio. €
412,30 Mio. €
377,20 Mio. €
316,00 Mio. €
256,40 Mio. €
214,85 Mio. €
209,65 Mio. €
165,10 Mio. €
148,25 Mio. €
148,00 Mio. €
135,53 Mio. €
118,40 Mio. €
101,95 Mio. €
100,80 Mio. €
91,50 Mio. €
85,65 Mio. €
41,60 Mio. €
38,95 Mio. €


In [36]:
# Mio. Euros wegschneiden

for wert in beträge[2:][::2]:
    print(wert.text.replace('Mio. €', ''))

827,80 
412,30 
377,20 
316,00 
256,40 
214,85 
209,65 
165,10 
148,25 
148,00 
135,53 
118,40 
101,95 
100,80 
91,50 
85,65 
41,60 
38,95 


In [37]:
# Kommas müssen noch weg

for wert in beträge[2:][::2]:
    print(wert.text.replace('Mio. €', '').replace(',','.'))

827.80 
412.30 
377.20 
316.00 
256.40 
214.85 
209.65 
165.10 
148.25 
148.00 
135.53 
118.40 
101.95 
100.80 
91.50 
85.65 
41.60 
38.95 


In [38]:
# Umwandeln in Floats damit der Computer diese als Zahlen versteht.

for wert in beträge[2:][::2]:
    print(float(wert.text.replace('Mio. €', '').replace(',','.')))

827.8
412.3
377.2
316.0
256.4
214.85
209.65
165.1
148.25
148.0
135.53
118.4
101.95
100.8
91.5
85.65
41.6
38.95


## Liste der Vereine mit jener des Werts zusammenführen

In [41]:
teamwertliste = []
for team, wert in zip(vereine, beträge[2:][::2]):
    team = team.text
    wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
    minidict = {'Team': team,
                'Wert': wert}
    teamwertliste.append(minidict)

In [42]:
pd.DataFrame(teamwertliste)

,Team,Wert
0,Bayern München,827.80
1,Bor. Dortmund,412.30
2,Bay. Leverkusen,377.20
3,RB Leipzig,316.00
4,FC Schalke 04,256.40
5,TSG Hoffenheim,214.85
6,Bor. M'gladbach,209.65
7,VfB Stuttgart,165.10
8,VfL Wolfsburg,148.25
9,E. Frankfurt,148.00


# Jedes einzelne Jahr besuchen

In [43]:
# Am Schluss bei der url einfach das Jahr auswechseln zeigt eine alte Saison an
# https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2016
# Ziel: die Urls bauen für die Jahre 2010-2019

jahre = list(range(2010,2019))

In [44]:
# url so abändern damit das Jahr fehlt

url = "https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id="

In [45]:
for jahr in jahre:
    print(jahr)
    print(url+str(jahr))

2010
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2010
2011
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2011
2012
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2012
2013
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2013
2014
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2014
2015
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2015
2016
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2016
2017
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2017
2018
https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2018


## Alles zusammenfügen

In [48]:
allejahre=[]
url = url = "https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id="
jahre = list(range(2010,2019))

# Ein For- Loop in einem For-Loop bauen

for jahr in jahre:
    url = url+str(jahr)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    # Damit das läuft muss es eingerückt sein, weil der untere Block Bezüge zum oberen hat
    for team, wert in zip(vereine, beträge[2:][::2]):
        team = team.text
        wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
        minidict = {'Team': team,
                'Wert': wert,
               'Saison': str(jahr)}
        allejahre.append(minidict)


In [49]:
pd.DataFrame(allejahre)

,Saison,Team,Wert
0,2010,Bayern München,827.80
1,2010,Bor. Dortmund,412.30
2,2010,Bay. Leverkusen,377.20
3,2010,RB Leipzig,316.00
4,2010,FC Schalke 04,256.40
5,2010,TSG Hoffenheim,214.85
6,2010,Bor. M'gladbach,209.65
7,2010,VfB Stuttgart,165.10
8,2010,VfL Wolfsburg,148.25
9,2010,E. Frankfurt,148.00
